In [ ]:
%AddDeps com.databricks spark-csv_2.10 1.4.0 --transitive

In [ ]:
%AddJar http://localhost:8888/nbextensions/declarativewidgets/urth-widgets.jar

In [ ]:
import urth.widgets._
initWidgets

In [ ]:
%%html
<link rel="import" href="urth_components/urth-viz-table/urth-viz-table.html" is="urth-core-import">
<link rel="import" href="urth_components/urth-viz-bar/urth-viz-bar.html" is="urth-core-import">
<link rel="import" href="urth_components/paper-input/paper-input.html" is='urth-core-import' package='PolymerElements/paper-input'>
<link rel="import" href="urth_components/paper-spinner/paper-spinner.html" is='urth-core-import' package='PolymerElements/paper-spinner'>

#### Spark DataFrame

In [ ]:
val rawFares = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").option("inferSchema", "true").load("demos/tour/fares.csv")
val fares = rawFares.select(rawFares.columns.map( col => rawFares.col(col).as(col.trim())):_*)

In [ ]:
%%html
<template id="t" is="dom-bind">
    <urth-core-dataframe id="d" ref="fares" value="{{df}}" busy="{{busy}}">
        <urth-core-query-filter disabled="{{!vid}}">vendor_id = '{{vid}}'</urth-core-query-filter>
        <urth-core-query-filter disabled="{{!ptype}}">payment_type = '{{ptype}}'</urth-core-query-filter>
        <urth-core-query-group by="{{gCol}}">
            <urth-core-query-agg op="sum" col="tolls_amount"></urth-core-query-agg>
        </urth-core-query-group>
        <urth-core-query-group by="sum(total_amount)" direction="desc">
    </urth-core-dataframe>
    <paper-input label="By" value="{{gCol}}"></paper-input>
    <paper-input label="Vendor ID" value="{{vid}}"></paper-input>
    <paper-input label="Payment Type" value="{{ptype}}"></paper-input>
    <paper-spinner active="{{busy}}"></paper-spinner>
    <urth-viz-table datarows="{{df.data}}" columns="{{df.columns}}"></urth-viz-table>
    <urth-viz-bar datarows="{{df.data}}"></urth-viz-bar>
</template>

#### Taxi demo on Scala

In [ ]:
%%html
<link rel="import" href="urth_components/paper-dropdown-menu/paper-dropdown-menu.html" is='urth-core-import' package='PolymerElements/paper-dropdown-menu'>
<link rel="import" href="urth_components/paper-menu/paper-menu.html" is='urth-core-import' package='PolymerElements/paper-menu'>
<link rel="import" href="urth_components/paper-item/paper-item.html" is='urth-core-import' package='PolymerElements/paper-item'>
<link rel="import" href="urth_components/paper-button/paper-button.html" is='urth-core-import' package='PolymerElements/paper-button'>
<link rel="import" href="urth_components/paper-card/paper-card.html" is='urth-core-import' package='PolymerElements/paper-card'>
<link rel="import" href="urth_components/paper-slider/paper-slider.html" is='urth-core-import' package='PolymerElements/paper-slider'>

<style is="custom-style">
    urth-viz-bar::shadow .nv-x .tick text {
        display: none;
    }
</style>

In [ ]:
import urth.widgets.WidgetChannels.channel
channel("faresChan").set("paymentOptions", List(("All", ""), ("Cash", "CSH"), ("Credit","CRD")))

In [ ]:
%%html
<template id="earnings" is="urth-core-bind" channel="faresChan">
    <urth-core-dataframe id="df" ref="fares" value="{{top}}" limit="{{limit}}" busy="{{busy}}" refresh-timeout="30000">
        <urth-core-query-filter disabled="{{!payType}}">payment_type = '{{payType}}'</urth-core-query-filter>
        <urth-core-query-group by="medallion">
            <urth-core-query-agg op="sum" col="total_amount"></urth-core-query-agg>
        </urth-core-query-group>
        <urth-core-query-sort by="sum(total_amount)" direction="{{topOrBottom}}"></urth-core-query-sort>
    </urth-core-dataframe>
    <paper-card style="width: 100%;" heading="Earners" elevation="1">
        <div class="card-content">
            <paper-spinner style="float:right;" active="{{busy}}"></paper-spinner>
            <paper-dropdown-menu label="Select Payment Type" noink>
                <paper-menu class="dropdown-content" selected="{{payType}}" attr-for-selected="value">
                    <template is="dom-repeat" items="[[paymentOptions]]">
                        <paper-item value="[[item.1]]">[[item.0]]</paper-item>
                    </template>
                </paper-menu>
            </paper-dropdown-menu>
            <paper-dropdown-menu label="Top or Bottom Earners" noink>
                <paper-menu class="dropdown-content" selected="{{topOrBottom}}" attr-for-selected="value">
                    <paper-item value="desc">Top</paper-item>
                    <paper-item value="asc">Bottom</paper-item>
                </paper-menu>
            </paper-dropdown-menu>

            <urth-viz-bar datarows="{{top.data}}" selection="{{sel}}" multi-select></urth-viz-bar>
            <div class="center horizontal layout">
                <div>Limit</div>
                <paper-slider min="5" max="50" step="5" pin snap value="{{limit}}"></paper-slider>
            </div>
        </div>
    </paper-card>
</template>